In [38]:
# Standard Imports 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [44]:
# Loadin gAPI Credentials
with open(r'C:\Users\zachd\.secret\yelp_api.json') as f: 
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

In [51]:
# Setting the API call parameters
LOCATION = 'Little Rock,AR'
TERM = 'Mexican'

# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = r"Data\results_in_progess_mexican_littlerock.json"
JSON_FILE

'Data\\results_in_progess_mexican_littlerock.json'

In [56]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f"[i] {JSON_FILE} exists. Deleting file.")           
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:

            print(f'[!] {JSON_FILE} already exists.')
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [58]:
## create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists = True)
## Load previous results and use len of results for offest
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

## Setting the offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# Using the yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term = TERM, location = LOCATION, offset = n_results)

# How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.clear
n_pages = math.ceil((results['total']-n_results) / results_per_page)
n_pages

[i] Data\results_in_progess_mexican_littlerock.json not found. Saving empty list to new file.
- 0 previous results found.


11

In [59]:
for i in tqdm_notebook(range(1, n_pages + 1)):
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)

    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
    
    # Using n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)
    # Appending the results to the previous results
    previous_results.extend(results['businesses'])
    # displaying (previous_results)
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)

    time.sleep(0.2)

  0%|          | 0/11 [00:00<?, ?it/s]

In [60]:
## Load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hiexEBd3sj7F351kwLhTKA,taqueria-el-palenque-little-rock,Taqueria El Palenque,https://s3-media4.fl.yelpcdn.com/bphoto/iY1u84...,False,https://www.yelp.com/biz/taqueria-el-palenque-...,186,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.7595502051828, 'longitude': -9...",[delivery],$,"{'address1': '9501 N Rodney Parham Rd', 'addre...",+15013120045,(501) 312-0045,5488.926460
1,xKsmbbj5ojsJk8_u2RmSTQ,la-tapatia-taqueria-and-birria-little-rock,La Tapatia Taqueria & Birria,https://s3-media4.fl.yelpcdn.com/bphoto/EwHjUv...,False,https://www.yelp.com/biz/la-tapatia-taqueria-a...,54,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.6710973360345, 'longitude': -9...",[delivery],$,"{'address1': '8912 Stagecoach Rd', 'address2':...",+15014552423,(501) 455-2423,9343.415459
2,rN_t-3eIBq4zibUXBzhK1w,la-hacienda-mexican-restaurant-little-rock,La Hacienda Mexican Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/_F4BIo...,False,https://www.yelp.com/biz/la-hacienda-mexican-r...,100,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.7608, 'longitude': -92.3091199}",[delivery],$,"{'address1': '3024 Cantrell Rd', 'address2': '...",+15016610600,(501) 661-0600,4959.671393
3,O7JSL7mFPzPWmh0_flpfTQ,heights-taco-and-tamale-little-rock,Heights Taco & Tamale,https://s3-media1.fl.yelpcdn.com/bphoto/RkFqGf...,False,https://www.yelp.com/biz/heights-taco-and-tama...,339,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",4.0,"{'latitude': 34.77, 'longitude': -92.33853}",[delivery],$$,"{'address1': '5805 Kavanaugh Blvd', 'address2'...",+15013134848,(501) 313-4848,5194.285361
4,IhQcF2m9RLOv1pbowr4Vnw,local-lime-little-rock,Local Lime,https://s3-media3.fl.yelpcdn.com/bphoto/gsf5c5...,False,https://www.yelp.com/biz/local-lime-little-roc...,393,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.76798700216534, 'longitude': -...",[delivery],$$,"{'address1': '17815 Chenal Pkwy', 'address2': ...",+15014482226,(501) 448-2226,12101.320338


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
203,Hb9tQCOnaeb7TVFNVS5RgA,ihop-little-rock-4,IHOP,https://s3-media2.fl.yelpcdn.com/bphoto/mbHZSd...,False,https://www.yelp.com/biz/ihop-little-rock-4?ad...,66,"[{'alias': 'breakfast_brunch', 'title': 'Break...",2.0,"{'latitude': 34.74661, 'longitude': -92.40333}","[pickup, delivery]",$,"{'address1': '11601 Hermitage Rd', 'address2':...",+15019547737,(501) 954-7737,6444.737326
204,4dbYTq3wTBf5mjQ8lEJJaA,red-lobster-north-little-rock,Red Lobster,https://s3-media3.fl.yelpcdn.com/bphoto/0fIlcX...,False,https://www.yelp.com/biz/red-lobster-north-lit...,69,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 34.79298405, 'longitude': -92.230...","[pickup, delivery]",$$,"{'address1': '3707 McCain Blvd', 'address2': '...",+15017534000,(501) 753-4000,12584.226555
205,9O8wv0kKIUY8iMCccGZtpA,ihop-north-little-rock-5,IHOP,https://s3-media1.fl.yelpcdn.com/bphoto/nW4Y2A...,False,https://www.yelp.com/biz/ihop-north-little-roc...,62,"[{'alias': 'tradamerican', 'title': 'American ...",2.0,"{'latitude': 34.79347, 'longitude': -92.22294}","[pickup, delivery]",$$,"{'address1': '4225 Warden Rd', 'address2': '',...",+15017719700,(501) 771-9700,13137.686199
206,mhzsYtzPpaWKCyKQJaYe9Q,jasons-deli-north-little-rock,Jason's Deli,https://s3-media4.fl.yelpcdn.com/bphoto/DBHaiy...,False,https://www.yelp.com/biz/jasons-deli-north-lit...,55,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.5,"{'latitude': 34.790220974891, 'longitude': -92...","[pickup, delivery]",$,"{'address1': '4209 E McCain Blvd', 'address2':...",+15019457700,(501) 945-7700,12997.787111
207,FkZ-Id7rYygqeyBTn0w7HQ,taco-bell-cabot,Taco Bell,https://s3-media4.fl.yelpcdn.com/bphoto/ubGotm...,False,https://www.yelp.com/biz/taco-bell-cabot?adjus...,12,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.0,"{'latitude': 34.977069, 'longitude': -92.015963}",[],$,"{'address1': '1430 W Locust', 'address2': '', ...",+15018430186,(501) 843-0186,40781.486163


In [61]:
# Dropping duplicates in the final_df
final_df = final_df.drop_duplicates(subset = 'id')
final_df.duplicated(subset = 'id').sum()

0

In [62]:
## Save the final results to a compressed csv
final_df.to_csv(r'Data\final_results_NY_pizza.csv.gz', compression='gzip', index=False)

In [63]:
# Checking if exported csv file exists
df = pd.read_csv(r'Data\final_results_NY_pizza.csv.gz', compression='gzip')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hiexEBd3sj7F351kwLhTKA,taqueria-el-palenque-little-rock,Taqueria El Palenque,https://s3-media4.fl.yelpcdn.com/bphoto/iY1u84...,False,https://www.yelp.com/biz/taqueria-el-palenque-...,186,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.7595502051828, 'longitude': -9...",['delivery'],$,"{'address1': '9501 N Rodney Parham Rd', 'addre...",1.501312e+10,(501) 312-0045,5488.926460
1,xKsmbbj5ojsJk8_u2RmSTQ,la-tapatia-taqueria-and-birria-little-rock,La Tapatia Taqueria & Birria,https://s3-media4.fl.yelpcdn.com/bphoto/EwHjUv...,False,https://www.yelp.com/biz/la-tapatia-taqueria-a...,54,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.6710973360345, 'longitude': -9...",['delivery'],$,"{'address1': '8912 Stagecoach Rd', 'address2':...",1.501455e+10,(501) 455-2423,9343.415459
2,rN_t-3eIBq4zibUXBzhK1w,la-hacienda-mexican-restaurant-little-rock,La Hacienda Mexican Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/_F4BIo...,False,https://www.yelp.com/biz/la-hacienda-mexican-r...,100,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.7608, 'longitude': -92.3091199}",['delivery'],$,"{'address1': '3024 Cantrell Rd', 'address2': '...",1.501661e+10,(501) 661-0600,4959.671393
3,O7JSL7mFPzPWmh0_flpfTQ,heights-taco-and-tamale-little-rock,Heights Taco & Tamale,https://s3-media1.fl.yelpcdn.com/bphoto/RkFqGf...,False,https://www.yelp.com/biz/heights-taco-and-tama...,339,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",4.0,"{'latitude': 34.77, 'longitude': -92.33853}",['delivery'],$$,"{'address1': '5805 Kavanaugh Blvd', 'address2'...",1.501313e+10,(501) 313-4848,5194.285361
4,IhQcF2m9RLOv1pbowr4Vnw,local-lime-little-rock,Local Lime,https://s3-media3.fl.yelpcdn.com/bphoto/gsf5c5...,False,https://www.yelp.com/biz/local-lime-little-roc...,393,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.76798700216534, 'longitude': -...",['delivery'],$$,"{'address1': '17815 Chenal Pkwy', 'address2': ...",1.501448e+10,(501) 448-2226,12101.320338
